<a href="https://colab.research.google.com/github/werd0n4/hand-gesture-classification/blob/master/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries

---


In [49]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [50]:
# !ls

In [51]:
import pandas as pd
import numpy as np
import seaborn as sns

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from tensorflow.keras import initializers

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import os
import cv2
import glob
import random
from matplotlib import pyplot as plt

# Constant parameters

---



In [52]:
horiz_x = 800  
vert_y = 600
#drive paths
#
imgs_train_path = '/content/drive/My Drive/Colab Notebooks/Rozszerzony_dataset/Train/'
imgs_test_path = '/content/drive/My Drive/Colab Notebooks/Rozszerzony_dataset/Test/'
#local paths windows
#
# imgs_train_path = 'C:\\Users\\Werdon\\Google Drive\\Colab Notebooks\\Rozszerzony_dataset\\Train'
# imgs_test_path = 'C:\\Users\\Werdon\\Google Drive\\Colab Notebooks\\Rozszerzony_dataset\\Train'
#local paths manjaro
#
# imgs_train_path = '/home/werdon4/Rozszerzony_dataset/Train'
# imgs_test_path = '/home/werdon4/Rozszerzony_dataset/Test'


In [53]:

class_names = {
    0: "1",
    1: "2",
    2: "3",
    3: "4",
    4: "5",
    5: "A",
    6: "B",
    7: "C",
    8: "D",
    9: "E",
    10: "F",
    11: "G",
    12: "H",
    13: "I",
    14: "K",
    15: "L",
    16: "M",
    17: "N",
    18: "O",
    19: "P",
    20: "R",
    21: "S",
    22: "T",
    23: "U",
    24: "W",
    25: "Y",
    26: "Z"
}


# Auxiliary functions

---


## Image resize

In [54]:

def resize(path):
    img_counter = 0

    for dirname in os.listdir(path): 
        for filename in os.listdir(os.path.join(path, dirname)):
            image_path = os.path.join(path, dirname, filename)
            img = cv2.imread(image_path)
            resized_img = cv2.resize(img, (horiz_x, vert_y))
            cv2.imwrite(image_path, resized_img)
            img_counter += 1
    
    print('Images in set: ' + str(img_counter))


## Image size sanity check

In [55]:

def sanity_check(path):
    counter = 0

    for dirname in os.listdir(path): 
        for filename in os.listdir(os.path.join(path, dirname)):
            image_path = os.path.join(path, dirname, filename)
            img = cv2.imread(image_path)
            if img.shape != (horiz_x, vert_y, 3):
                counter += 1

    print('Sanity result: ' + str(counter))

In [56]:

def show_img(index, X, Y):
    # plt.imshow(X[index])
    plt.imshow(cv2.cvtColor(X[index],cv2.COLOR_BGR2RGB).astype('float32'))
    plt.show()
    nmb = int(np.where(Y[index] == 1)[0])
    print("On image: " + class_names[nmb])

## Load dataset

In [57]:

def load_dataset():
    trainlist = glob.glob(f'{imgs_train_path}/*/*')
    testlist = glob.glob(f'{imgs_test_path}/*/*')
    X_train = np.array( [np.array(cv2.normalize(cv2.imread(fname), None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)) for fname in trainlist] )
    X_test = np.array( [np.array(cv2.normalize(cv2.imread(fname), None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)) for fname in testlist] )

    # Y_train = np.array([np.zeros(27) for fname in trainlist])
    Y_train = np.array([0 for fname in trainlist])
    for i,fname in enumerate(trainlist):
        # print(fname)
        img_id = fname.split('/')[7]##7 if windows 4 if manjaro
        img_id = img_id.split('_')[0]
        # Y_train[i][img_id] = 1
        Y_train[i] = img_id


    # Y_test = np.array([np.zeros(27) for fname in testlist])
    Y_test = np.array([0 for fname in testlist])
    for i,fname in enumerate(testlist):
        img_id = fname.split('/')[7]
        img_id = img_id.split('_')[0]
        # Y_test[i][img_id] = 1
        Y_test[i] = img_id

    return X_train, Y_train, X_test, Y_test

# load_dataset()


## Create network model

In [58]:

def create_model():
    model = Sequential()
    # kernel_initializer = initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None)
    kernel_initializer=None 
    activ_func = 'relu'
    kernel_regularizer=None
    activity_regularizer=None

    # CONVOLUTIONAL LAYER - 1
    model.add(Conv2D(
        filters=6, 
        kernel_size=(5,5), 
        input_shape=(vert_y, horiz_x, 3), 
        activation=activ_func,
        # kernel_initializer=kernel_initializer
        # kernel_regularizer=kernel_regularizer
        # activity_regularizer=activity_regularizer
    ))
    # CONVOLUTIONAL LAYER - 1
    model.add(Conv2D(
        filters=6, 
        kernel_size=(5,5), 
        input_shape=(vert_y, horiz_x, 3), 
        activation=activ_func,
        # kernel_initializer=kernel_initializer
        # kernel_regularizer=kernel_regularizer
        # activity_regularizer=activity_regularizer
    ))

    # POOLING LAYER - 1
    model.add(MaxPool2D(
        pool_size=(2,2),
        strides=(2,2)
    ))

    # CONVOLUTIONAL LAYER - 2
    model.add(Conv2D(
        filters=16, 
        kernel_size=(3,3), 
        activation=activ_func,
        # kernel_initializer=kernel_initializer
        # kernel_regularizer=kernel_regularizer
        # activity_regularizer=activity_regularizer
    ))
    # CONVOLUTIONAL LAYER - 2
    model.add(Conv2D(
        filters=16, 
        kernel_size=(3,3), 
        activation=activ_func,
        # kernel_initializer=kernel_initializer
        # kernel_regularizer=kernel_regularizer
        # activity_regularizer=activity_regularizer
    ))

    # POOLING LAYER - 2
    model.add(MaxPool2D(
        pool_size=(2,2),
        strides=(2,2)
    ))

    # CONVOLUTIONAL LAYER - 3
    model.add(Conv2D(
        filters=32, 
        kernel_size=(3,3), 
        activation=activ_func,
        # kernel_initializer=kernel_initializer
        # kernel_regularizer=kernel_regularizer
        # activity_regularizer=activity_regularizer
    ))

    # POOLING LAYER - 3
    model.add(MaxPool2D(
        pool_size=(2,2),
        strides=(2,2)
    ))

    # CONVOLUTIONAL LAYER - 4
    model.add(Conv2D(
        filters=60, 
        kernel_size=(3,3), 
        activation=activ_func,
        # kernel_initializer=kernel_initializer
        # kernel_regularizer=kernel_regularizer
        # activity_regularizer=activity_regularizer
    ))

    # POOLING LAYER - 4
    model.add(MaxPool2D(
        pool_size=(2,2),
        strides=(2,2)
    ))

    # CONVOLUTIONAL LAYER - 5
    model.add(Conv2D(
        filters=120, 
        kernel_size=(3,3), 
        activation=activ_func,
        # kernel_initializer=kernel_initializer
        # kernel_regularizer=kernel_regularizer
        # activity_regularizer=activity_regularizer
    ))

    # POOLING LAYER - 5
    model.add(MaxPool2D(
        pool_size=(2,2),
        strides=(2,2)
    ))

    ######## FLATTEN ########
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    # model.add(Dense(64, activation='relu'))
    model.add(Dense(27, activation='softmax'))

    model.compile(
        loss='categorical_crossentropy',
        optimizer='adam', 
        metrics=['accuracy']
    )

    return model


# Loading dataset

---


## Resize images and do sanity check

In [59]:
# Uncomment before first run on dataset 
# resize(imgs_train_path)
# sanity_check(imgs_train_path)
# resize(imgs_test_path)
# sanity_check(imgs_test_path)


In [60]:
# X_train, Y_train, X_test, Y_test = load_dataset()

In [61]:
# One hot encoding
# Y_cat_train = to_categorical(Y_train, 27)
# Y_cat_test = to_categorical(Y_test, 27)

## Data augumentation

## Initializing ImageDataGenerator class

In [62]:
image_gen = ImageDataGenerator(rotation_range=5, # rotate the image 20 degrees
                               width_shift_range=0.05, # Shift the pic width by a max of 5%
                               height_shift_range=0.05, # Shift the pic height by a max of 5%
                            #    rescale=1.1, # Rescale the image by normalzing it.
                               shear_range=0.05, # Shear means cutting away part of the image (max 10%)
                               zoom_range=0.05, # Zoom in by 10% max
                               fill_mode='nearest' # Fill in missing pixels with the nearest filled value
                                )
                              

## Augumentation sample result

In [63]:
#augumentation sample result

#original img
# nmb = random.randint(0, 296)
# my_hand = X_test[nmb]
# show_img(nmb, X_test, Y_cat_test)

# #generated img
# gen_img = image_gen.random_transform(my_hand)
# print("Generated image")
# plt.imshow(cv2.cvtColor(gen_img,cv2.COLOR_BGR2RGB).astype('float32'))
# plt.show()

# comparison = my_hand == gen_img
# equal_arrays = comparison.all()
# print("Images are equal?: " + str(equal_arrays))

## Image shape and batch size initialization

In [64]:
image_shape = (horiz_x, vert_y, 3)
batch_size = 16

print(imgs_train_path)
print(imgs_test_path)

/content/drive/My Drive/Colab Notebooks/Rozszerzony_dataset/Train/
/content/drive/My Drive/Colab Notebooks/Rozszerzony_dataset/Test/


## Initializing test and train image geneartors

In [65]:
train_image_gen = image_gen.flow_from_directory(imgs_train_path,
                                               target_size=image_shape[:2],
                                                color_mode='rgb',
                                               batch_size=batch_size,
                                               class_mode='categorical')

Found 1561 images belonging to 27 classes.


In [66]:
test_image_gen = image_gen.flow_from_directory(imgs_test_path,
                                               target_size=image_shape[:2],
                                               color_mode='rgb',
                                               batch_size=batch_size,
                                               class_mode='categorical',shuffle=False)

Found 297 images belonging to 27 classes.


In [67]:
#train_image_gen.class_indices

# Create and train model

---


In [68]:
CNN = create_model()
CNN.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_14 (Conv2D)           (None, 596, 796, 6)       456       
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 592, 792, 6)       906       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 296, 396, 6)       0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 294, 394, 16)      880       
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 292, 392, 16)      2320      
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 146, 196, 16)      0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 144, 194, 32)     

In [69]:
early_stop = EarlyStopping(monitor='val_loss', patience=3)
#Y_cat_train.shape

## Fit model

In [70]:
# CNN.fit(X_train, Y_cat_train, epochs=12, validation_data=(X_test, Y_cat_test), batch_size=batch_size, callbacks=[early_stop])

## Fit model with data augumentation

In [72]:
results = CNN.fit(train_image_gen, epochs=32, validation_data=test_image_gen, callbacks=[early_stop])

Epoch 1/32
98/98 [==============================] - 172s 2s/step - loss: 3.2955 - accuracy: 0.0468 - val_loss: 3.2825 - val_accuracy: 0.0370
Epoch 2/32
98/98 [==============================] - 168s 2s/step - loss: 3.3017 - accuracy: 0.0455 - val_loss: 3.2942 - val_accuracy: 0.0505
Epoch 3/32
98/98 [==============================] - 170s 2s/step - loss: 3.2765 - accuracy: 0.0570 - val_loss: 3.2712 - val_accuracy: 0.0438
Epoch 4/32
98/98 [==============================] - 167s 2s/step - loss: 3.2279 - accuracy: 0.0679 - val_loss: 3.1805 - val_accuracy: 0.0875
Epoch 5/32
98/98 [==============================] - 168s 2s/step - loss: 3.1145 - accuracy: 0.0730 - val_loss: 2.9901 - val_accuracy: 0.1145
Epoch 6/32
98/98 [==============================] - 177s 2s/step - loss: 2.9712 - accuracy: 0.1262 - val_loss: 2.8780 - val_accuracy: 0.1515
Epoch 7/32
98/98 [==============================] - 172s 2s/step - loss: 2.8546 - accuracy: 0.1461 - val_loss: 2.7942 - val_accuracy: 0.1549
Epoch 8/32
98

## Saving model to file

In [ ]:
CNN.save('hand_gesture_classifier.h5')

# Results

---


In [ ]:
metrics = pd.DataFrame(CNN.history.history)
metrics

## Single image prediction

In [ ]:
# nmb = 2
# my_hand = X_test[nmb]
# show_img(nmb, X_test, Y_cat_test)

In [ ]:
# my_hand = my_hand.reshape(1, vert_y, horiz_x, 3)
# result = CNN.predict_classes(my_hand)
# # result = (CNN.predict(my_hand) > 0.5).astype("int32")
# result = int(result)
# print('CNN says: ' + class_names[result])

## Predictions for individual classes

In [ ]:
predictions = CNN.predict_classes(X_test)

In [ ]:
print(classification_report(Y_test,predictions))